# To geocode the HDB Projects

In [3]:
import pandas as pd

In [24]:
df1 = pd.read_csv('Applicationrate_HDB_19-25.csv')
df1 =df1[['project','plh','loccate']]
df1 = df1.groupby('project', as_index=False).first()

In [25]:
df = df1
df1

,project,plh,loccate
0,Verandah @ Kallang,Yes,Mature
1,Alexandra Peaks,Yes,Mature
2,Alexandra Vale / Havelock Hillside,Yes,Mature
3,Bartley Beacon / ParkView @ Bidadari,No,Mature
4,Bartley GreenRise / ParkEdge @ Bidadari / Alka...,No,Mature
...,...,...,...
108,Woodlands North Verge,No,Standard
109,Woodlands South Plains,No,Non-mature
110,Yio Chu Kang Beacon,No,Mature
111,Yishun Beacon,No,Non-mature


In [26]:
# Split 'project' by '/', remove extra spaces, explode into separate rows
df['project'] = df['project'].str.split('/')
df['project'] = df['project'].apply(lambda x: [p.strip() for p in x])  # remove extra spaces
df = df.explode('project', ignore_index=True)

In [31]:

df['lat'] = 0
df['lon'] = 0
df

,project,plh,loccate,lat,lon
0,Verandah @ Kallang,Yes,Mature,0,0
1,Alexandra Peaks,Yes,Mature,0,0
2,Alexandra Vale,Yes,Mature,0,0
3,Havelock Hillside,Yes,Mature,0,0
4,Bartley Beacon,No,Mature,0,0
...,...,...,...,...,...
144,Woodlands South Plains,No,Non-mature,0,0
145,Yio Chu Kang Beacon,No,Mature,0,0
146,Yishun Beacon,No,Non-mature,0,0
147,Yishun Boardwalk,No,Non-mature,0,0


In [40]:
def geocode(address):

    import requests
      
    url = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=" + address + "&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        
    headers = {"Authorization": "Bearer **********************"}
        
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        if response.json()['found'] > 0:
            
            data = response.json()['results'][0]
            print("Found address: ", address, data['ADDRESS'])
            lat = data['LATITUDE']
            lon = data['LONGITUDE']
            return lat, lon

In [42]:
response.json()['results'][0]

{'SEARCHVAL': 'VERANDAH@KALLANG',
 'BLK_NO': '118',
 'ROAD_NAME': 'KALLANG AIRPORT WAY',
 'BUILDING': 'VERANDAH@KALLANG',
 'ADDRESS': '118 KALLANG AIRPORT WAY VERANDAH@KALLANG SINGAPORE 390118',
 'POSTAL': '390118',
 'X': '32645.8003478795',
 'Y': '32496.1407564096',
 'LATITUDE': '1.31015772162034',
 'LONGITUDE': '103.875063350295'}

In [43]:
for i in df.index:
    print(i)
    address = df['project'][i]
    print(address)
    lat, lon = geocode(address)
    df.at[i, 'lat'] = lat
    df.at[i, 'lon'] = lon
    print(df.at[i, 'lat'], df.at[i, 'lon'])

0
Verandah @ Kallang
Found address:  Verandah @ Kallang 118 KALLANG AIRPORT WAY VERANDAH@KALLANG SINGAPORE 390118
1.31015772162034 103.875063350295
1
Alexandra Peaks


C:\Users\THW_9\AppData\Local\Temp\ipykernel_23556\3352000867.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.31015772162034' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'lat'] = lat
C:\Users\THW_9\AppData\Local\Temp\ipykernel_23556\3352000867.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '103.875063350295' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'lon'] = lon


Found address:  Alexandra Peaks 140C ALEXANDRA ROAD ALEXANDRA PEAKS SINGAPORE 153140
1.29151981991682 103.822576649518
2
Alexandra Vale
Found address:  Alexandra Vale ALEXANDRA VALE
1.28960133225885 103.818276680258
3
Havelock Hillside
Found address:  Havelock Hillside 1 HILLSIDE TERRACE SINGAPORE 548896
1.35730126173098 103.877726260223
4
Bartley Beacon
Found address:  Bartley Beacon 222 MOUNT VERNON ROAD BARTLEY BEACON SINGAPORE 360222
1.34088960796369 103.879139775745
5
ParkView @ Bidadari
Found address:  ParkView @ Bidadari PARKVIEW @ BIDADARI
1.3384536176892 103.874785626258
6
Bartley GreenRise
Found address:  Bartley GreenRise 216A BIDADARI PARK DRIVE BARTLEY GREENRISE SINGAPORE 361216
1.34304143501032 103.877363026119
7
ParkEdge @ Bidadari
Found address:  ParkEdge @ Bidadari 227 BARTLEY WALK PARKEDGE @ BIDADARI SINGAPORE 360227
1.34024749735901 103.876330655485
8
Alkaff Breeze
Found address:  Alkaff Breeze 119 ALKAFF CRESCENT ALKAFF BREEZE SINGAPORE 340119
1.33674489248516 103.8

In [45]:
df.to_csv('Applicationrate_HDB_19-25_geocoded.csv', index=False)